# BagBoo

## CART (Regression)

In [227]:
import numpy as np
import sklearn.datasets as ds
import pylab as pl
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
import random as r
import sys

def MSE(x, y):
    return ((x - y)**2).sum() / float(len(x))

#calculate variance of set
def variance_calculate(X):
#     n_points = X.shape[0]
#     var = 0
    
#     for i1 in xrange(n_points):
#         for i2 in xrange(i1 + 1, n_points):
#             var += (X[i1] - X[i2]) ** 2;
#             print i1, i2, var
    
#     print "calculate varianse of ", X, (1.0 / (2 * n_points ** 2)) * var
    
#     return (1.0 / n_points ** 2) * var
#     print X
    return np.var(X)
    
# search best split in X of split_feature
def search_best_split_of_feat(X, Y, split_feature, orig_var, min_samples_leaf):
    best_gain = 0
    checked_split_value = []
    best_split_value = None
    res_X_l = None
    res_X_r = None
    res_Y_l = None
    res_Y_r = None
    
    for split_value in X[:, split_feature]:
        if split_value in checked_split_value:
            continue
        checked_split_value.append(split_value)
#         print np.where(X[:, split_feature] < split_value)
        X_l = X[np.where(X[:, split_feature] < split_value)]
        X_r = X[np.where(X[:, split_feature] >= split_value)]
        Y_l = Y[np.where(X[:, split_feature] < split_value)]
        Y_r = Y[np.where(X[:, split_feature] >= split_value)]
#         print "split_value: ", split_value
#         print X_l, X_r, Y_l, Y_r
#         print X_l.shape[0]
#         print X_r.shape[0]
        if X_l.shape[0] >= min_samples_leaf and X_r.shape[0] > min_samples_leaf:
            var_l = variance_calculate(Y_l)
            var_r = variance_calculate(Y_r)
            
#             print orig_var, var_l, var_r,\
#                 (orig_var - (len(Y_l) / float(len(Y))) * var_l - (len(Y_r) / float(len(Y))) * var_r), best_gain
#             if (orig_var - var_l - var_r) > best_gain:
            if (orig_var - (len(Y_l) / float(len(Y))) * var_l - (len(Y_r) / float(len(Y))) * var_r) > best_gain:
                best_gain = orig_var - (len(Y_l) / float(len(Y))) * var_l - (len(Y_r) / float(len(Y))) * var_r
                best_split_value = split_value
                res_X_l = X_l
                res_X_r = X_r
                res_Y_l = Y_l
                res_Y_r = Y_r
    
    return best_gain, best_split_value, res_X_l, res_X_r, res_Y_l, res_Y_r

class Tree:
    def __init__(self, split_feature = None, split_val = None,\
                 left = None, right = None, n_points = None, depth = 10, min_samples_leaf = 1):
        self.split_feature = split_feature
        self.split_val = split_val
        self.left = left
        self.right = right
        self.data = None
        self.n_points = n_points
        self.depth = depth
        self.min_samples_leaf = min_samples_leaf
    
    #return MSE
    def score(self, X, Y):
        predicted = self.predict(X)
        MSE = ((predicted - Y)**2).sum() / float(len(X))
        print "MSE: ", MSE

    def predict_one(self, sample):
        if self.data is not None:
            return self.data

        if sample[self.split_feature] >= self.split_value:
            return self.right.predict_one(sample)
        else:
            return self.left.predict_one(sample)

    def predict(self, X):
        predicted = []
        for sample in X:
#             print sample
            predicted.append(self.predict_one(sample))
        return np.asarray(predicted)
            
    #building tree
    def fit (self, X, Y):
#         print "start_fitting:", X, Y
        self.n_points = X.shape[0]
        if self.n_points <= self.min_samples_leaf or self.depth == 0:
#             print "N_POINTS: ", self.n_points
            self.data = np.mean(Y)
            return self
        
        best_gain = 0
        res_split_value = 0
        res_split_feature = None
        res_X_r = None
        res_X_l = None
        res_Y_l = None
        res_Y_r = None
        orig_var = variance_calculate(Y)
        
        n_feat = X.shape[1]
        for feat in xrange(n_feat):
#             print 
#             print "Analusys of feature ", feat
            gain, split_value, X_l , X_r, Y_l, Y_r = search_best_split_of_feat(X, Y, feat, orig_var, self.min_samples_leaf)
            if gain > best_gain:
                best_gain = gain
                res_split_value = split_value
                res_split_feature = feat
                res_X_r = X_r
                res_X_l = X_l
                res_Y_l = Y_l
                res_Y_r = Y_r
        
#         print "BEST SPLIT: "
#         print res_split_feature, res_split_value
#         print res_X_l, res_X_r, res_Y_l, res_Y_r
#         print 
            
        if best_gain > 0:
            self.split_feature = res_split_feature
            self.split_value = res_split_value
            if self.depth is None:
                self.left = Tree(min_points_leaf = self.min_points_leaf).fit(res_X_l, res_Y_l)
                self.right = Tree(min_points_leaf = self.min_points_leaf).fit(res_X_r, res_Y_r)
            else:
                self.left = Tree(depth = self.depth - 1, min_samples_leaf = self.min_samples_leaf).fit(res_X_l, res_Y_l)
                self.right = Tree(depth = self.depth - 1, min_samples_leaf = self.min_samples_leaf).fit(res_X_r, res_Y_r)
                
        else:
            self.n_points = len(Y)
#             print "N_POINTS: ", self.n_points
#             print X
#             print Y
            self.data = np.mean(Y)
        return self
            
    def print_tree(self):
        if self.data != None:
            print str(self.data)
        else:
            print str(self.split_feature) + ": " + str(self.split_value) + "?"
            print "\t T->"
            if (self.right):
                self.right.print_tree()
            print "\t F->"
            if self.left:
                self.left.print_tree()
        

class Gradient_Boosting:
    def __init__(self, n_estimators=10, shrinkage=0.05, max_depth=10, min_samples_leaf=1):
        self.estimators_list = []
        self.n_estimators = n_estimators
        self.shrinkage = shrinkage
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        
    def fit(self, X, Y):
        self.estimators_list = []
        first_estimator = Tree(depth= self.max_depth, min_samples_leaf= self.min_samples_leaf).fit(X, Y)
        self.estimators_list.append(first_estimator)
        
        current_predict = first_estimator.predict(X)
#         sys.stderr.write('\rLearning estimator number: 0'+ "/" + str(self.n_estimators))
#         print  "\tLearning estimator number: 0 ; MSE error on train dataset: ", MSE(current_predict, Y)
        
        for i in xrange(1, self.n_estimators):
            
            antigrad = Y - current_predict
            
            new_estimator = Tree(depth=self.max_depth, min_samples_leaf=self.min_samples_leaf)
            new_estimator = new_estimator.fit(X, antigrad)
#             new_estimator.print_tree()
            
#             print set(antigrad)
#             print new_estimator.predict(X)[:10]
            current_predict += self.shrinkage * new_estimator.predict(X)
            
#             if i % 10 == 0:
#                 print "\tLearning estimator number: ", i,\
#                         "; MSE error on train dataset: ", MSE(current_predict, Y)
            
            sys.stderr.write('\rLearning estimator number: '+ str(i)+"/" + str(self.n_estimators) \
                             + "; MSE error on train dataset: " + str(MSE(current_predict, Y)))
            self.estimators_list.append(new_estimator)
    
    def predict(self, X):
        y =  self.estimators_list[0].predict(X)
#         print len(y)
        for estimator in self.estimators_list[1:]:
            y += estimator.predict(X) * self.shrinkage
#             print MSE(y, Y)
        return y

#n_bag -count of bagging iteration
#n_boo - count of tree in Gradien Boosting
#max_depth - max depth of trees TODO:dinamic depth of trees
#min_samples_leaf
#bagging_ratio -cnt of samples (in percent), which using for bagging interation
#RSM and Bagging without replacement
class BagBoo:
    def __init__ (self, n_boo = 10, n_bag = 10, bagging_ratio = 0.1, rsm_ratio = 1, max_depth = 10,\
                  min_samples_leaf = 1, shrinkage = 0.1):
        self.n_boo = n_boo
        self.n_bag = n_bag
        self.bagging_ratio = bagging_ratio
        self.rsm_ratio = rsm_ratio
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.boosting_list = []
        self.shrinkage = shrinkage
        
    def fit(self, X, Y, verbose = 0, X_test = None, Y_test = None):
        cur_sum_predict = 0 
        cur_sum_train = 0
        error_statistic = []
        rsm_cnt = int(self.rsm_ratio * X.shape[1])
        bagging_cnt = int(self.bagging_ratio * X.shape[0])
        print "features in RMS: ", rsm_cnt
        print "samples in bagging: ", bagging_cnt
        for bag_iter in xrange(self.n_bag):
            sys.stderr.write('\rIteration of bagging:'+str(bag_iter) + "/" + str(self.n_bag))
#            print "Iteration of bagging: "+ str(bag_iter) + "/" + str(self.n_bag)
            shuffle_idx_bagging = range(X.shape[0])
            shuffle_idx_rsm = range(X.shape[1])
            
            r.shuffle(shuffle_idx_bagging)
            r.shuffle(shuffle_idx_rsm)
            shuffle_idx_bagging = shuffle_idx_bagging[:bagging_cnt]
            shuffle_idx_rsm = shuffle_idx_rsm[:rsm_cnt]

#             print shuffle_idx_bagging, shuffle_idx_rsm
            X_bag = X[shuffle_idx_bagging][:, shuffle_idx_rsm]
            Y_bag = Y[shuffle_idx_bagging]
#             print "sgs", X_bag, Y_bag
            
            new_boosting = Gradient_Boosting(n_estimators= self.n_boo, max_depth=self.max_depth,\
                                             min_samples_leaf= self.min_samples_leaf, shrinkage = self.shrinkage)
            new_boosting.fit(X_bag, Y_bag)
            
            self.boosting_list.append(new_boosting)
            
            if verbose:
                cur_sum_predict += new_boosting.predict(X_test)
                cur_sum_train += new_boosting.predict(X)
                error_test =  MSE(cur_sum_predict / float(len(self.boosting_list)), Y_test)
                error_train =  MSE(cur_sum_train / float(len(self.boosting_list)), Y)
                print "MSE on test Dataset:", error_test, "Iteration of Bagging:", bag_iter, "/", self.n_bag
                print "MSE on train Dataset:", error_train, "Iteration of Bagging:", bag_iter, "/", self.n_bag
                error_statistic.append(error_test)
               
        return error_statistic    

    def predict(self, X):
        y = np.array([0.0] * X.shape[0])
        for boosting in self.boosting_list:
            y += boosting.predict(X)
        return y / float(self.n_bag)


In [228]:
f = [1, 2, 3]
r.shuffle(f)
print f

[2, 1, 3]


In [229]:
X = np.array([[1, 0, 5], [2, 3, 5], [2, 3, 4], [2, 9, 1], [2, 9, 4], [1, 2, 1]]*10)
Y = np.array([1, 0, 3, 6, 4, 1]*10)

In [230]:
test = BagBoo(bagging_ratio=0.5, rsm_ratio=0.8).fit(X, Y)

features in RMS:  2
samples in bagging:  3
[1, 3, 0] [2, 1]
sgs [[5 3]
 [1 9]
 [5 0]] [0 6 1]
[3, 1, 2] [0, 1]
sgs [[2 9]
 [2 3]
 [2 3]] [6 0 3]
[0, 4, 5] [2, 1]
sgs [[5 0]
 [4 9]
 [1 2]] [1 4 1]
[0, 1, 4] [0, 2]
sgs [[1 5]
 [2 5]
 [2 4]] [1 0 4]
[5, 0, 2] [2, 0]
sgs [[1 1]
 [5 1]
 [4 2]] [1 1 3]
[0, 3, 4] [2, 0]
sgs [[5 1]
 [1 2]
 [4 2]] [1 6 4]
[4, 2, 3] [0, 2]
sgs [[2 4]
 [2 4]
 [2 1]] [4 3 6]
[3, 2, 1] [1, 2]
sgs [[9 1]
 [3 4]
 [3 5]] [6 3 0]
[5, 0, 2] [2, 0]
sgs [[1 1]
 [5 1]
 [4 2]] [1 1 3]
[2, 1, 0] [2, 1]
sgs [[4 3]
 [5 3]
 [5 0]] [3 0 1]


Learning estimator number: 9/10; MSE error on train dataset: 0.0604284960788

In [226]:
x = np.array([[1, 2, 3], [4, 5, 6], [3, 4, 5]])
print x[np.array([0,2])][:, np.array([1, 2])]
# [: np.array([2, 3])]

[[2 3]
 [4 5]]


In [59]:
tree_1 = Tree().fit(X, Y) 

In [197]:
tree_1.predict([[1, 2, 3],[8,-2,6]])

array([ 1.25,  1.25])

In [198]:
print tree_1.print_tree()

1: 9?
	 T->
5.0
	 F->
1.25
None


### Avto-dataset

In [215]:
avto = pd.read_csv("./data/auto-mpg.data.csv", sep='\t',header=None)
avto = avto.values
print avto.shape
print avto

(392, 9)
[[18.0 8 307.0 ..., 70 1 'chevrolet chevelle malibu']
 [15.0 8 350.0 ..., 70 1 'buick skylark 320']
 [18.0 8 318.0 ..., 70 1 'plymouth satellite']
 ..., 
 [32.0 4 135.0 ..., 82 1 'dodge rampage']
 [28.0 4 120.0 ..., 82 1 'ford ranger']
 [31.0 4 119.0 ..., 82 1 'chevy s-10']]


In [216]:
avto_data = avto[:, 1:8]
avto_target = avto[:, 0]

In [217]:
print avto_data
print avto_target[:5]

[[8 307.0 130.0 ..., 12.0 70 1]
 [8 350.0 165.0 ..., 11.5 70 1]
 [8 318.0 150.0 ..., 11.0 70 1]
 ..., 
 [4 135.0 84.0 ..., 11.6 82 1]
 [4 120.0 79.0 ..., 18.6 82 1]
 [4 119.0 82.0 ..., 19.4 82 1]]
[18.0 15.0 18.0 16.0 17.0]


In [218]:
%%time
tree_avto = Tree(depth=4).fit(avto_data[:350], avto_target[:350])

CPU times: user 612 ms, sys: 4 ms, total: 616 ms
Wall time: 685 ms


In [219]:
tree_avto.score(avto_data[350:392], avto_target[350:392])

MSE:  24.0325897867


In [220]:
sk_avto = DecisionTreeRegressor(max_depth=4).fit(avto_data[:350], avto_target[:350])
skl_predict = sk_avto.predict(avto_data[350:392])
print ((skl_predict - avto_target[350:392])**2 / float(len(skl_predict))).sum()

24.4742424835


In [221]:
%%time
gb_avto = Gradient_Boosting(n_estimators=100, max_depth=2, shrinkage=0.1)    
gb_avto.fit(avto_data[:300], avto_target[:300])

print "MSE Error on train dataset: ", MSE(gb_avto.predict(avto_data[:300]), avto_target[:300])
print "MSE Error on test dataset: ", MSE(gb_avto.predict(avto_data[300:]), avto_target[300:])

Learning estimator number: 27/100; MSE error on train dataset: 3.85458861375

KeyboardInterrupt: 

In [51]:
%%time
skl_gb_avto = GradientBoostingRegressor(criterion='mse', min_impurity_split=0,\
                                        learning_rate=0.1, max_depth=2, n_estimators=100)
skl_gb_avto.fit(avto_data[:300], avto_target[:300])
print "MSE Error on train dataset: ", MSE(skl_gb_avto.predict(avto_data[:300]), avto_target[:300])
print "MSE Error on test dataset: ", MSE(skl_gb_avto.predict(avto_data[300:]), avto_target[300:])

MSE Error on train dataset:  1.96854048843
MSE Error on test dataset:  31.7097021516
CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 35.4 ms


In [281]:
print ((skl_predict - avto_target[300:392])**2 / float(len(skl_predict))).sum()

35.7244551349


In [153]:
%%time
bagboo_avto = BagBoo(n_boo=20, n_bag=50, bagging_ratio=0.3, max_depth=3)    
bagboo_avto.fit(avto_data[:300], avto_target[:300])

Iteration of bagging: 0/50


Learning estimator number: 19/20; MSE error on train dataset: 1.51237822924

Iteration of bagging: 1/50


Learning estimator number: 19/20; MSE error on train dataset: 1.57575090254

Iteration of bagging: 2/50


...

Learning estimator number: 19/20; MSE error on train dataset: 1.20035670834

Iteration of bagging: 49/50


Learning estimator number: 17/20; MSE error on train dataset: 1.08085784396

CPU times: user 1min 23s, sys: 1.17 s, total: 1min 24s
Wall time: 1min 23s


Learning estimator number: 19/20; MSE error on train dataset: 1.0078936038

In [95]:
bagboo_avto.predict(avto_data[:300])[:10]

array([ 16.42715978,  14.65455132,  15.62765702,  15.65619808,
        16.47238784,  13.79927187,  13.72700477,  13.74249449,
        13.52174017,  14.19534903])

In [96]:
print MSE(bagboo_avto.predict(avto_data[:300]), avto_target[:300])
print MSE(bagboo_avto.predict(avto_data[300:392]), avto_target[300:392])

3.23658112515
37.1373884278


## House-dataset

In [293]:
house = pd.read_csv("./data/housing.data.csv", sep='\t',header=None)
house = house.values
print house.shape
print house
house_data = house[:, :13]
house_target = house[:, 13]
print house_data[:10]
print house_target[:10]

(506, 14)
[[  6.32000000e-03   1.80000000e+01   2.31000000e+00 ...,   3.96900000e+02
    4.98000000e+00   2.40000000e+01]
 [  2.73100000e-02   0.00000000e+00   7.07000000e+00 ...,   3.96900000e+02
    9.14000000e+00   2.16000000e+01]
 [  2.72900000e-02   0.00000000e+00   7.07000000e+00 ...,   3.92830000e+02
    4.03000000e+00   3.47000000e+01]
 ..., 
 [  6.07600000e-02   0.00000000e+00   1.19300000e+01 ...,   3.96900000e+02
    5.64000000e+00   2.39000000e+01]
 [  1.09590000e-01   0.00000000e+00   1.19300000e+01 ...,   3.93450000e+02
    6.48000000e+00   2.20000000e+01]
 [  4.74100000e-02   0.00000000e+00   1.19300000e+01 ...,   3.96900000e+02
    7.88000000e+00   1.19000000e+01]]
[[  6.32000000e-03   1.80000000e+01   2.31000000e+00   0.00000000e+00
    5.38000000e-01   6.57500000e+00   6.52000000e+01   4.09000000e+00
    1.00000000e+00   2.96000000e+02   1.53000000e+01   3.96900000e+02
    4.98000000e+00]
 [  2.73100000e-02   0.00000000e+00   7.07000000e+00   0.00000000e+00
    4.6900

In [355]:
%%time
tree_house = Tree(depth=4).fit(house_data[:300], house_target[:300])
tree_house.score(house_data[300:392], house_target[300:392])

MSE:  55.02303951
CPU times: user 552 ms, sys: 0 ns, total: 552 ms
Wall time: 558 ms


In [356]:
sk_house = DecisionTreeRegressor(max_depth=4).fit(house_data[:300], house_target[:300])
skl_predict = sk_house.predict(house_data[300:392])
print ((skl_predict - house_target[300:392])**2 / float(len(skl_predict))).sum()

55.02303951


In [367]:
%%time
gb_house = Gradient_Boosting(n_estimators=101, max_depth=3, shrinkage=0.1)    
gb_house.fit(house_data[:400], house_target[:400])

print "MSE Error on train dataset: ", MSE(gb_house.predict(house_data[:400]), house_target[:400])
print "MSE Error on test dataset: ", MSE(gb_house.predict(house_data[400:506]), house_target[400:506])

Number of learning estimator: 0; MSE error on train dataset:  2558.21747067
Number of learning estimator:  10 ; MSE error on train dataset:  1744.14453063
Number of learning estimator:  20 ; MSE error on train dataset:  1308.94369915
Number of learning estimator:  30 ; MSE error on train dataset:  1037.5467406
Number of learning estimator:  40 ; MSE error on train dataset:  848.176763415
Number of learning estimator:  50 ; MSE error on train dataset:  718.57484164
Number of learning estimator:  60 ; MSE error on train dataset:  615.749565531
Number of learning estimator:  70 ; MSE error on train dataset:  528.983190297
Number of learning estimator:  80 ; MSE error on train dataset:  473.463570747
Number of learning estimator:  90 ; MSE error on train dataset:  423.31122444
Number of learning estimator:  100 ; MSE error on train dataset:  383.96837126
MSE Error on train dataset:  383.96837126
MSE Error on test dataset:  2019.41281341
CPU times: user 55.3 s, sys: 48 ms, total: 55.4 s
Wal

In [364]:
%%time
skl_gb_house = GradientBoostingRegressor(criterion='mse', min_impurity_split=0)
skl_gb_house.fit(house_data[:400], house_target[:400])
print "MSE Error on train dataset: ", MSE(skl_gb_house.predict(house_data[:400]), house_target[:400])
print "MSE Error on test dataset: ", MSE( skl_gb_house.predict(house_data[400:506]), house_target[400:506])

MSE Error on train dataset:  320.259854388
MSE Error on test dataset:  671.613810248
CPU times: user 104 ms, sys: 0 ns, total: 104 ms
Wall time: 104 ms


## Computer Hardware

In [108]:
machine = pd.read_csv("./data/machine.data.csv", sep=',',header=None)
machine = machine.values
print machine.shape
print machine
machine_data = machine[:, 2:9]
machine_target = machine[:, 9]
print machine_data[:10]
print machine_target[:10]

(209, 10)
[['adviser' '32/60' 125 ..., 128 198 199]
 ['amdahl' '470v/7' 29 ..., 32 269 253]
 ['amdahl' '470v/7a' 29 ..., 32 220 253]
 ..., 
 ['sratus' '32' 125 ..., 14 52 41]
 ['wang' 'vs-100' 480 ..., 0 67 47]
 ['wang' 'vs-90' 480 ..., 0 45 25]]
[[125 256 6000 256 16 128 198]
 [29 8000 32000 32 8 32 269]
 [29 8000 32000 32 8 32 220]
 [29 8000 32000 32 8 32 172]
 [29 8000 16000 32 8 16 132]
 [26 8000 32000 64 8 32 318]
 [23 16000 32000 64 16 32 367]
 [23 16000 32000 64 16 32 489]
 [23 16000 64000 64 16 32 636]
 [23 32000 64000 128 32 64 1144]]
[199 253 253 253 132 290 381 381 749 1238]


In [109]:
%%time
tr_house = Tree().fit(machine_data[:150], machine_target[:150])

N_POINTS:  149
N_POINTS:  1
CPU times: user 172 ms, sys: 12 ms, total: 184 ms
Wall time: 168 ms


In [113]:
%%time
gb_tr = Gradient_Boosting(max_depth=3)    
gb_tr.fit(machine_data[:30], machine_target[:30])

N_POINTS:  16
N_POINTS:  12
N_POINTS:  1
N_POINTS:  1
Number of learning estimator: 0
MSE error on train dataset:  73723.84375
Number of learning estimator:  1
N_POINTS:  30
0.0
set([-7.75, 46.25, 4.6875, 83.25, 33.6875, 8.6875, 14.6875, -15.3125, 0.0, 174.25, 9.6875, -7.3125, -6.3125, -136.75, -3.3125, -8.3125, -1.3125, -2.3125, -178.75, -82.75, -89.75, -9.3125, -74.75]) [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
MSE error on train dataset:  73723.84375
Number of learning estimator:  2
N_POINTS:  30
0.0
set([-7.75, 46.25, 4.6875, 83.25, 33.6875, 8.6875, 14.6875, -15.3125, 0.0, 174.25, 9.6875, -7.3125, -6.3125, -136.75, -3.3125, -8.3125, -1.3125, -2.3125, -178.75, -82.75, -89.75, -9.3125, -74.75]) [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
MSE error on train dataset:  73723.84375
Number of learning estimator:  3
N_POINTS:  30
0.0
set([-7.75, 46.25, 4.6875, 83.25, 33.6875, 8.6875, 14.6875, -15.3125, 0.0, 174.25, 9.6875, -7.3125, -6.3125, -136.75, -3.3125, -8.3125, -1.3125, -2.3125, -178.7

## Spam

In [161]:
spam_train = np.loadtxt("./data/spam.train.txt")
spam_test = np.loadtxt("./data/spam.test.txt")
spam_train_data = spam_train[0::, 1::]
spam_train_target = spam_train[0::, 0]
spam_test_data = spam_test[0::, 1::]
spam_test_target = spam_test[0::, 0]
print spam_train.shape
print spam_test.shape

(7093, 103)
(10056, 103)


In [162]:
# print spam_test_data[:1]
# print spam_test_target[:455]
# print spam_train_data[:1]
# print spam_train_target[:455]
print np.sum([1 for s in spam_train_target if s == 0])
print np.sum([1 for s in spam_test_target if s == 0])

2970
2033


In [168]:
%%time
gb_spam = Gradient_Boosting(n_estimators=10, max_depth=3, shrinkage=0.1)    
gb_spam.fit(spam_train_data, spam_train_target)
# print "MSE Error on train dataset: ", MSE(gb_spam.predict(house_data[:400]), house_target[:400])
print "MSE Error on test dataset: ", MSE(gb_spam.predict(spam_test_data), spam_test_target)

TypeError: fit() got an unexpected keyword argument 'verbose'

In [118]:
%%time
bagboo_spam = BagBoo(n_boo=20, n_bag=50, bagging_ratio=0.02, max_depth=2)    
bagboo_spam.fit(spam_train_data, spam_train_target)

print "MSE Error on test dataset: ", MSE(bagboo_spam.predict(spam_test_data), spam_test_target)

7093
Number of learning estimator: 0; MSE error on train dataset:  0.0819392272764
Number of learning estimator:  10 ; MSE error on train dataset:  0.0475617430457
7093
Number of learning estimator: 0; MSE error on train dataset:  0.0851488004344
Number of learning estimator:  10 ; MSE error on train dataset:  0.049674908376
7093
Number of learning estimator: 0; MSE error on train dataset:  0.0808008154593
Number of learning estimator:  10 ; MSE error on train dataset:  0.0456885511967
7093
Number of learning estimator: 0; MSE error on train dataset:  0.105370596725
Number of learning estimator:  10 ; MSE error on train dataset:  0.0589344066274
7093
Number of learning estimator: 0; MSE error on train dataset:  0.0832367290717
Number of learning estimator:  10 ; MSE error on train dataset:  0.0517909094027
7093
Number of learning estimator: 0; MSE error on train dataset:  0.0785564450533
Number of learning estimator:  10 ; MSE error on train dataset:  0.0488745509803
7093
Number of lea

In [172]:
%%time
skl_gb_spam = GradientBoostingRegressor(criterion='mse', min_impurity_split=0, verbose=2, learning_rate=0.01)
skl_gb_spam.fit(spam_train_data, spam_train_target)
# print "MSE Error on train dataset: ", MSE(skl_gb_house.predict(house_data[:400]), house_target[:400])
print "MSE Error on test dataset: ", MSE( skl_gb_house.predict(spam_test_data), spam_test_target)

      Iter       Train Loss   Remaining Time 
         1           0.2404            8.65s
         2           0.2375            8.86s
         3           0.2347            8.38s
         4           0.2319            8.14s
         5           0.2291            7.62s
         6           0.2264            7.27s
         7           0.2238            7.20s
         8           0.2212            6.87s
         9           0.2186            6.49s
        10           0.2161            6.32s
        11           0.2137            6.13s
        12           0.2112            5.87s
        13           0.2089            5.66s
        14           0.2066            5.46s
        15           0.2043            5.43s
        16           0.2020            5.29s
        17           0.1999            5.24s
        18           0.1977            5.19s
        19           0.1956            5.24s
        20           0.1935            5.21s
        21           0.1914            5.16s
        2

In [231]:
x = np.array([1, 2, 3])
print x**2

[1 4 9]
